# Scrapping riot api :D

## Setting up
Please obtain ur own api key from riot games dev website

In [ ]:
#set up
import requests
import json
import time
import pandas as pd
import random

apiKey = "ur api key here"

#for reproducability
seed = 99
random.seed(seed)

## Getting the sum id of players in a reigon using league v4

In [ ]:
def summPuller(division,tier,page):
    import pandas as pd
    
    url = "https://na1.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/{}/{}?page={}&api_key={}".format(division, tier, page, apiKey)
    listProfiles = requests.get(url).json()
    profileNum = len(listProfiles)
    listSumID = []
    csvName = division + "summID.csv"
    for profile in range(0,profileNum):
        listSumID.append(listProfiles[profile]["summonerId"])
        
    df = pd.DataFrame(listSumID,columns = ["ID"])
    df.to_csv(csvName,mode = 'a')

divs = ["IRON", "GOLD", "DIAMOND"]
tiers = ["I","II","III","IV"]
for div in divs:
    time.sleep(5)
    for i in tiers:
        for page in range(1,5):
            time.sleep(2)
            summPuller(div,i,page)

## Now we change those encrypted ids to locate the actual id of the player using summoner v4

In [ ]:
ironSummID = pd.read_csv("IRONsummID.csv")
goldSummID = pd.read_csv("GOLDsummID.csv")
diaSummID = pd.read_csv("DIAMONDsummID.csv")
ironAccs = []
goldAccs = []
diaAccs = []

def idPuller(id,accList):
    url = "https://na1.api.riotgames.com/lol/summoner/v4/summoners/{}?api_key={}".format(id, apiKey)
    info = requests.get(url).json()
    accList.append(info["puuid"])

iron100 = ironSummID.sample(n = 100, random_state = seed)
gold100 = goldSummID.sample(n = 100, random_state = seed)
dia100 = diaSummID.sample(n = 100, random_state = seed)

ironSummIDList = iron100["ID"]
goldSummIDList = gold100["ID"]
diaSummIDList = dia100["ID"]

# sleep time is needed to respect riot's rate limit
i = 0
for id in ironSummIDList:
    i+= 1
    print(i)
    idPuller(id, ironAccs)
time.sleep(120)
i = 0
for id in goldSummIDList:
    i+= 1
    print(i)
    idPuller(id, goldAccs)
time.sleep(120)
i = 0
for id in diaSummIDList:
    i+= 1
    print(i)
    idPuller(id, diaAccs)
time.sleep(120)

### Converting to csv once again

In [ ]:
df = pd.DataFrame(ironAccs, columns = ["puuid"])
df.to_csv('ironAccID.csv',mode = 'a')

df = pd.DataFrame(goldAccs, columns = ["puuid"])
df.to_csv('goldAccID.csv',mode = 'a')

df = pd.DataFrame(diaAccs, columns = ["puuid"])
df.to_csv('diaAccID.csv',mode = 'a')


## Now we pull matches from those players using match v5

In [ ]:
ironMatchIDList = []
goldMatchIDList = []
diaMatchIDList = []

def matchPuller(id, matchList):
    url = "https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{}/ids?type=ranked&start=0&count=5&api_key={}".format(id, apiKey)
    response = requests.get(url).json()
    for i in range(0,5):        
            match_id = response[i]
            matchList.append(match_id)

i = 0
for id in ironAccs:
    i+= 1
    if i == 100:
         time.sleep(120)
    print(i)
    matchPuller(id, ironMatchIDList)
time.sleep(120)

i = 0
for id in goldAccs:
    i+= 1
    if i == 100:
         time.sleep(120)
    print(i)
    matchPuller(id, goldMatchIDList)
time.sleep(120)

i = 0
for id in diaAccs:
    i+= 1
    if i == 100:
         time.sleep(120)
    print(i)
    matchPuller(id, diaMatchIDList)


### Converting to csv

In [ ]:
df = pd.DataFrame(ironMatchIDList, columns = ["matchID"])
df.to_csv('ironMatches.csv',mode = 'a')

df = pd.DataFrame(goldMatchIDList, columns = ["matchID"])
df.to_csv('goldMatches.csv',mode = 'a')

df = pd.DataFrame(diaMatchIDList, columns = ["matchID"])
df.to_csv('diaMatches.csv',mode = 'a')


## Function for getting overall data of the match using match v5

In [ ]:
def getMatchData(matchID):
    url = f"https://americas.api.riotgames.com/lol/match/v5/matches/{matchID}?api_key={apiKey}"
    matchResponse = requests.get(url).json()
    match_data = []

    teams = matchResponse["info"]["teams"]
    participants = matchResponse["info"]["participants"]

    game_duration = matchResponse["info"]["gameDuration"]
    match_id = matchResponse["metadata"]["matchId"]

    for team in teams:
        team_id = team["teamId"]
        is_winner = team["win"]

        team_stats = {
            "match_id": match_id,
            "team_id": team_id,
            "is_winner": is_winner,
            "game_duration": game_duration,
            "tower_kills": team["objectives"]["tower"]["kills"],
            "dragon_kills": team["objectives"]["dragon"]["kills"],
            "baron_kills": team["objectives"]["baron"]["kills"],
        }

        team_participants = [p for p in participants if p["teamId"] == team_id]
        total_kills = sum(p["kills"] for p in team_participants)
        total_assists = sum(p["assists"] for p in team_participants)
        total_deaths = sum(p["deaths"] for p in team_participants)
        total_damage_dealt = sum(p["totalDamageDealtToChampions"] for p in team_participants)
        total_gold_earned = sum(p["goldEarned"] for p in team_participants)
        total_minions_killed = sum(p["totalMinionsKilled"] for p in team_participants)
        total_time_cc_dealt = sum(p["timeCCingOthers"] for p in team_participants)
        total_time_played = sum(p["timePlayed"] for p in team_participants)
        had_afk_teammates = any(p.get("gameEndedInEarlySurrender", False) for p in team_participants)
        total_scuttle_crab_kills = sum(p["challenges"].get("scuttleCrabKills", 0) for p in team_participants)
        total_ward_takedowns = sum(p["challenges"].get("wardsTakedown", 0) for p in team_participants)
        total_wards_placed = sum(p["wardsPlaced"] for p in team_participants)

        team_stats.update({
            "total_kills": total_kills,
            "total_assists": total_assists,
            "total_deaths": total_deaths,
            "total_damage_dealt": total_damage_dealt,
            "total_gold_earned": total_gold_earned,
            "total_minions_killed": total_minions_killed,
            "total_time_cc_dealt": total_time_cc_dealt,
            "total_time_played": total_time_played,
            "had_afk_teammates": had_afk_teammates,
            "total_scuttle_crab_kills": total_scuttle_crab_kills,
            "total_ward_takedowns": total_ward_takedowns,
            "total_wards_placed": total_wards_placed,
        })

        match_data.append(team_stats)

    return match_data

## Function for getting match data at 15 min mark using match v5

In [ ]:
import json
import pandas as pd
import requests

def get15min_data(matchID):
    url = f"https://americas.api.riotgames.com/lol/match/v5/matches/{matchID}/timeline?api_key={apiKey}"
    timeline_json = requests.get(url).json()
    match_id = timeline_json["metadata"]["matchId"]
    frames = timeline_json["info"]["frames"]

    team1_stats = {"match_id": match_id, "team_id": 100, "gold15": 0, "damage_done15": 0, "kills15": 0, "cs15": 0}
    team2_stats = {"match_id": match_id, "team_id": 200, "gold15": 0, "damage_done15": 0, "kills15": 0, "cs15": 0}

    if len(frames) < 15:
       return [team1_stats, team2_stats]
    

    frame_15 = frames[14]["participantFrames"]
    
    for participant_id, data in frame_15.items():
        participant_id = int(participant_id)
        team = 1 if participant_id <= 5 else 2
        
        gold = data["totalGold"]
        damage_done = data["damageStats"]["totalDamageDoneToChampions"]
        kills = data["damageStats"]["totalDamageTaken"]
        cs = data["minionsKilled"] + data["jungleMinionsKilled"]
        
        if team == 1:
            team1_stats["gold15"] += gold
            team1_stats["damage_done15"] += damage_done
            team1_stats["kills15"] += kills
            team1_stats["cs15"] += cs
        else:
            team2_stats["gold15"] += gold
            team2_stats["damage_done15"] += damage_done
            team2_stats["kills15"] += kills
            team2_stats["cs15"] += cs
    return [team1_stats, team2_stats]

### Conversion to csv

In [ ]:
t = pd.read_csv("ironMatches.csv")
ironMatchIDList = t['matchID'].tolist()
t = pd.read_csv("goldMatches.csv")
goldMatchIDList= t['matchID'].tolist()
t = pd.read_csv("diaMatches.csv")
diaMatchIDList= t['matchID'].tolist()

In [ ]:
ironMatchdata = []
ironMatchData15 = []

goldMatchdata = []
goldMatchData15 = []

diaMatchdata = []
diaMatchData15 = []

## Actually getting the match data and data at 15 min and combining

In [ ]:
i = 0
for id in ironMatchIDList:
    i+= 1
    if i % 80 == 0:
         time.sleep(120)
    ironMatchdata.extend(getMatchData(id))
    time.sleep(2)
    ironMatchData15.extend(get15min_data(id))
    time.sleep(2)
time.sleep(120)

ironDF = pd.DataFrame(ironMatchdata)
ironDF15 = pd.DataFrame(ironMatchData15)
ironComb = pd.merge(ironDF,ironDF15, on = ["match_id", "team_id"], how="inner")
ironComb.to_csv("ironCombData.csv", index=False)

In [ ]:
i = 0
for id in goldMatchIDList:
    i+= 1
    if i % 80 == 0:
         time.sleep(120)
    goldMatchdata.extend(getMatchData(id))
    time.sleep(2)
    goldMatchData15.extend(get15min_data(id))  
    time.sleep(2)
time.sleep(120)

goldDF = pd.DataFrame(goldMatchdata)
goldDF15 = pd.DataFrame(goldMatchData15)
goldComb = pd.merge(goldDF,goldDF15, on = ["match_id", "team_id"], how="inner")
goldComb.to_csv("goldCombData.csv", index=False)

In [ ]:
i = 0
for id in diaMatchIDList:
    i+= 1
    if i % 80 == 0:
         time.sleep(120)
    diaMatchdata.extend(getMatchData(id))
    time.sleep(2)
    diaMatchData15.extend(get15min_data(id)) 
    time.sleep(2)

diaDF = pd.DataFrame(diaMatchdata)
diaDF15 = pd.DataFrame(diaMatchData15)
diaComb = pd.merge(diaDF,diaDF15, on = ["match_id", "team_id"], how="inner")
diaComb.to_csv("dianCombData.csv", index=False)